# Load SageMaker GroundTruth annotation

## Load task manifest file mapping text sample IDs to text

In [ ]:
import json
from pathlib import Path

In [ ]:
manifest_path = Path("annotations/glue-dir-kbase-dev-sagemaker-ground-truth-labeling-clone/annotations/intermediate/1/annotations.manifest")

In [ ]:
assert manifest_path.is_file()

In [ ]:
index_to_text = {}

In [ ]:
with open(manifest_path, "rt") as fin:
    for line in fin:
        columns = line.split("\t")
        index = int(columns[0])
        text = "\t".join(columns[1:-1])
        _ = columns[-1]  # no idea what this column is
        assert index not in index_to_text
        index_to_text[index] = text

## Load annotation for given sample text IDs 

In [ ]:
annotations_dir = Path("annotations/glue-dir-kbase-dev-sagemaker-ground-truth-labeling-clone/annotations/worker-response/iteration-1/")

In [ ]:
assert annotations_dir.is_dir()

In [ ]:
for annotations_subdir in annotations_dir.iterdir():
    index = int(annotations_subdir.name)
    for i, annotations_file in enumerate(annotations_subdir.iterdir()):
        assert i == 0, f"found more than one annotation in {annotations_subdir}"
        with open(annotations_file, "rt") as fin:
            j = json.load(fin)
        break

In [ ]:
j

In [ ]:
index_to_text[index]

TODO:

- remove episode ID
- adjust text and tag positions
- load into spacy
- visualize for sanity checking